# **Azure AI Agents with Action Tools**

## Overview
This notebook demonstrates how to create and use AI agents with action tools in Azure AI Foundry. You'll learn how to enhance an AI agent's capabilities by integrating custom functions that can perform actions on behalf of the user, allowing the agent to interact with external systems and APIs.

## Learning Objectives
- Set up an Azure AI agent with custom function tools
- Create a weather assistant that can fetch weather information
- Define and implement custom Python functions as agent tools
- Create and manage conversation threads
- Process tool calls and handle their execution flow
- Submit tool outputs back to the agent
- Monitor run status and handle different action requirements

## Prerequisites
- An Azure account with access to Azure AI Foundry
- Appropriate environment variables in a `.env` file:
  - `PROJECT_CONNECTION_STRING`: Connection string for your Azure AI Foundry project
  - `chatModel`: The model to use for the agent (e.g., GPT-4o)

## Workflow
This notebook walks through the complete lifecycle of an AI agent with action tools:
1. Setting up the environment and client
2. Defining custom functions that the agent can call
3. Configuring system instructions for the agent
4. Creating the agent with the function tools
5. Managing conversation threads and messages
6. Running the agent and handling tool call execution
7. Processing responses that incorporate tool outputs
8. Cleaning up resources when finished

Follow along step by step to create your own action-enabled AI agent!

**Imports**

In [1]:
import os
import dotenv
dotenv.load_dotenv(".env")

True

**Create Client**

In [2]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

**Action Tools**

In [3]:
import json
from typing import Any, Callable, Set, Optional
    
def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """
    # Mock API response.
    mock_weather_data = {
        "New York": "Sunny, 25°C",
        "London": "Cloudy, 18°C",
        "Tokyo": "Rainy, 22°C",
        "Stockholm": "Snowy, -5°C",
    }
    
    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json

def check_network_status(location: str, service_type: Optional[str] = None) -> str:
    """
    Checks the status of telecommunications network in a specified location.
    
    :param location (str): The location to check network status for.
    :param service_type (str, optional): Type of service to check (e.g., "4G", "5G", "VoLTE").
                                         If not specified, returns status for all services.
    :return: Network status information as a JSON string.
    :rtype: str
    """
    # Mock network status data
    network_data = {
        "Stockholm": {
            "5G": "Operational - 98.7% coverage",
            "4G": "Operational - 99.9% coverage",
            "VoLTE": "Operational"
        },
        "Gothenburg": {
            "5G": "Partial outage - 85.3% coverage",
            "4G": "Operational - 99.5% coverage",
            "VoLTE": "Operational"
        },
        "New York": {
            "5G": "Operational - 92.1% coverage",
            "4G": "Operational - 99.7% coverage",
            "VoLTE": "Maintenance in progress"
        },
        "London": {
            "5G": "Operational - 90.5% coverage",
            "4G": "Operational - 99.8% coverage",
            "VoLTE": "Operational"
        }
    }
    
    # Check if the location exists in our mock data
    if location not in network_data:
        return json.dumps({"error": "Location not found in network database"})
    
    # Return data for the specific service type if provided
    if service_type:
        if service_type in network_data[location]:
            return json.dumps({
                "location": location,
                "service": service_type,
                "status": network_data[location][service_type],
                "timestamp": "2025-04-04T10:30:00Z"  # Using current date from context
            })
        else:
            return json.dumps({"error": f"Service type {service_type} not available in {location}"})
    
    # Return all service data for the location
    return json.dumps({
        "location": location,
        "services": network_data[location],
        "timestamp": "2025-04-04T10:30:00Z"  # Using current date from context
    })


# Statically defined user functions for fast reference
agent_functions: Set[Callable[..., Any]] = {
    fetch_weather,
    check_network_status
}


**System message**

In [4]:
system_message=(
    "You are an assistant with access to both weather information and telecommunications network status data. "
    "You can help with the following:\n"
    "1. Checking current weather conditions in various locations\n"
    "2. Providing network status information in different regions, including details about 4G, 5G, and VoLTE services\n\n"
    "For network status, you can check overall network conditions in a location or get specific information about "
    "a particular service like 5G coverage. Feel free to ask for clarification if the user's request is ambiguous."
)

**Create Agent**

In [5]:
from azure.ai.projects.models import FunctionTool

functions = FunctionTool(functions=agent_functions)

agent = project_client.agents.create_agent(
    name="my-action-tool-agent",
    model=os.getenv("chatModel"),
    instructions=system_message,
    tools=functions.definitions,
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_M40iRhAEXRxEi8sfZNL65H7m


**Create Thread**

In [6]:
thread = project_client.agents.create_thread()
agent = project_client.agents.get_agent(agent_id=agent.id)

**Adding Message**

In [7]:
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="How's the 5G network in Stockholm?",
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_BddcNZItT4CvWZCwRIscwBi5


**Running Agent with Thread**

In [8]:
import time
from azure.ai.projects.models import RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput

run = project_client.agents.create_run(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, ID: {run.id}")

while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
        # When the status is requires_action, your code is responsible for calling the function tools.
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        if not tool_calls: # If no tool calls are provided, cancel the run.
            print("No tool calls provided - cancelling run")
            project_client.agents.cancel_run(thread_id=thread.id, run_id=run.id)
            break

        tool_outputs = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredFunctionToolCall):
                try:
                    print(f"Executing tool call: {tool_call}")
                    output = functions.execute(tool_call)
                    # If the output is not a string, convert it to JSON string
                    if not isinstance(output, str):
                        output = json.dumps(output)
                    tool_outputs.append(
                        ToolOutput(
                            tool_call_id=tool_call.id,
                            output=output,
                        )
                    )
                except Exception as e:
                    print(f"Error executing tool_call {tool_call.id}: {e}")

        print(f"Tool outputs: {tool_outputs}")
        if tool_outputs:
            project_client.agents.submit_tool_outputs_to_run(
                thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
            )
    print(f"Current run status: {run.status}")
print(f"Run completed with status: {run.status}")

Created run, ID: run_aveJtZdbU8uPyTsliK5cWEAb
Current run status: in_progress
Current run status: in_progress
Current run status: in_progress
Executing tool call: {'id': 'call_Tx2nRAt55hGplg9zrZ8X9Wq8', 'type': 'function', 'function': {'name': 'check_network_status', 'arguments': '{"location":"Stockholm","service_type":"5G"}'}}
Tool outputs: [{'tool_call_id': 'call_Tx2nRAt55hGplg9zrZ8X9Wq8', 'output': '{"location": "Stockholm", "service": "5G", "status": "Operational - 98.7% coverage", "timestamp": "2025-04-04T10:30:00Z"}'}]
Current run status: requires_action
Current run status: completed
Run completed with status: completed


In [9]:
from utils.helper import pretty_print_thread_messages

messages = project_client.agents.list_messages(thread_id=thread.id)
pretty_print_thread_messages(messages)


👤 USER
------------------------------------------------------------


How's the 5G network in Stockholm?


🤖 ASSISTANT
------------------------------------------------------------


The 5G network in Stockholm is currently operational with 98.7% coverage.

**Adding new message**

In [10]:
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="what's the weather in Stockholm?",
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_fLZFmzvECLm7kDwdFGxwHs07



**Running Agent with Thread**

In [11]:
import time
from azure.ai.projects.models import RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput

run = project_client.agents.create_run(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, ID: {run.id}")

while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
        # When the status is requires_action, your code is responsible for calling the function tools.
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        if not tool_calls: # If no tool calls are provided, cancel the run.
            print("No tool calls provided - cancelling run")
            project_client.agents.cancel_run(thread_id=thread.id, run_id=run.id)
            break

        tool_outputs = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredFunctionToolCall):
                try:
                    print(f"Executing tool call: {tool_call}")
                    output = functions.execute(tool_call)
                    # If the output is not a string, convert it to JSON string
                    if not isinstance(output, str):
                        output = json.dumps(output)
                    tool_outputs.append(
                        ToolOutput(
                            tool_call_id=tool_call.id,
                            output=output,
                        )
                    )
                except Exception as e:
                    print(f"Error executing tool_call {tool_call.id}: {e}")

        print(f"Tool outputs: {tool_outputs}")
        if tool_outputs:
            project_client.agents.submit_tool_outputs_to_run(
                thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
            )
    print(f"Current run status: {run.status}")
print(f"Run completed with status: {run.status}")

Created run, ID: run_3mILps3TG4nB3cdJk5qHkagu

Executing tool call: {'id': 'call_q4zT5EbmIcL96qE5LBPIFL6e', 'type': 'function', 'function': {'name': 'fetch_weather', 'arguments': '{"location":"Stockholm"}'}}
Tool outputs: [{'tool_call_id': 'call_q4zT5EbmIcL96qE5LBPIFL6e', 'output': '{"weather": "Snowy, -5\\u00b0C"}'}]
Current run status: requires_action
Current run status: completed
Run completed with status: completed


In [12]:
from utils.helper import pretty_print_thread_messages

messages = project_client.agents.list_messages(thread_id=thread.id)
pretty_print_thread_messages(messages)


👤 USER
------------------------------------------------------------


How's the 5G network in Stockholm?


🤖 ASSISTANT
------------------------------------------------------------


The 5G network in Stockholm is currently operational with 98.7% coverage.


👤 USER
------------------------------------------------------------


what's the weather in Stockholm?


🤖 ASSISTANT
------------------------------------------------------------


The current weather in Stockholm is snowy with a temperature of -5°C.

**Delete Agent & Thread**

In [13]:
# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

# Delete Thread when done
project_client.agents.delete_thread(thread_id=thread.id)
print(f"thread: {thread.id} deleted")

Deleted agent

thread: thread_SvpMnldBhzd0ZZpynmvGWccW deleted
